# Libraries


# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project-automl-experiment'

experiment = Experiment(ws, experiment_name)

In [3]:
dataset = Dataset.get_by_name(ws, "default-ibk-dataset")
df = dataset.to_pandas_dataframe()

In [4]:
df.head()

,key_value,target,edad,sexo,est_cvl,sit_lab,ctd_hijos,flg_sin_email,ctd_veh,tip_lvledu,...,total_max_of_saldo_sum_per_month,total_min_of_saldo_min_per_month,total_mean_of_saldo_min_per_month,total_max_of_saldo_max_per_month,total_mean_of_saldo_max_per_month,total_min_of_condicion_min_per_month,total_mean_of_condicion_min_per_month,total_max_of_condicion_max_per_month,total_mean_of_condicion_max_per_month,total_mean_of_condicion_mean_per_month
0,0,0,-0.983651,1.0,4.0,2.0,0.0,1.0,0.0,0.0,...,6.829334,0.927141,0.936001,1.072672,1.014399,0,0.0,0,0.000,0.000000
1,1,0,0.000953,1.0,4.0,1.0,0.0,1.0,1.0,0.0,...,3.729381,0.927119,0.927238,0.938372,0.938051,0,0.0,0,0.000,0.000000
2,2,0,0.531124,0.0,4.0,1.0,0.0,1.0,0.0,7.0,...,2.849616,0.927119,0.927119,0.995254,0.964115,0,0.0,0,0.000,0.000000
3,3,0,-1.135129,0.0,4.0,1.0,0.0,1.0,0.0,7.0,...,4.076464,0.927209,0.928015,1.114270,1.104669,0,0.0,0,0.000,0.000000
4,4,1,-1.438084,0.0,4.0,1.0,0.0,1.0,0.0,0.0,...,11.229775,0.927220,0.928755,0.957766,0.952834,0,0.0,80,19.125,1.730303


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

amlcompute_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')

except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 1)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded............
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [10]:
project_folder = './capstone-project-automl'

In [11]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted',
    "enable_early_stopping": True,
    "n_cross_validations": 5,
    "max_cores_per_iteration": -1
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="target",
                             path = project_folder,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project-automl-experiment,AutoML_36449ed6-91de-49f6-b34b-deb0dad4489b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [13]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone-project-automl-experiment,AutoML_36449ed6-91de-49f6-b34b-deb0dad4489b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

{'runId': 'AutoML_36449ed6-91de-49f6-b34b-deb0dad4489b',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-11T05:49:02.853209Z',
 'endTimeUtc': '2021-07-11T06:37:42.732523Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project-automl-experiment","subscription_id":"e8d8d6b4-0477-45de-ad97-b824b737d6f3","resource_group":"myresourcegroup","workspace_name":"azudacityml01","region":"eastus2","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"A

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [14]:
#list(experiment.get_runs())[0].get_details()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
best_run, model = remote_run.get_output()

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [16]:
best_run.get_details()

{'runId': 'AutoML_36449ed6-91de-49f6-b34b-deb0dad4489b_30',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-07-11T06:31:32.150766Z',
 'endTimeUtc': '2021-07-11T06:37:41.272144Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Stack_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Stack_Ensemble__","objects":[{"module":"azureml.train.automl.stack_ensemble","class_name":"StackEnsemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-project-automl-experiment\',\'compute_target\':\'automl-cluster\',\'subscription_id\':\'e8d8d6b4-0477-45de-ad97-b824b737d6f3\',\'region\':\'eastus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_36449ed6-91de-49f6-b34b-deb0dad4489b_30","experiment_name":"capstone-project-automl-experiment","workspace_name":"az

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Usin

In [17]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":3,"CurrentNodeCount":3}',
 'mlflow.source.type': 'JOB',
 'mlflow.source.name': 'automl_driver.py',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True',
 'model_explain_run_id': 'AutoML_36449ed6-91de-49f6-b34b-deb0dad4489b_ModelExplain',
 'model_explanation': 'True'}

In [ ]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

In [18]:
best_run.properties

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Stack_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Stack_Ensemble__","objects":[{"module":"azureml.train.automl.stack_ensemble","class_name":"StackEnsemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'AUC_weighted\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone-project-automl-experiment\',\'compute_target\':\'automl-cluster\',\'subscription_id\':\'e8d8d6b4-0477-45de-ad97-b824b737d6f3\',\'region\':\'eastus2\',\'spark_service\':None}","ensemble_run_id":"AutoML_36449ed6-91de-49f6-b34b-deb0dad4489b_30","experiment_name":"capstone-project-automl-experiment","workspace_name":"azudacityml01","subscription_id":"e8d8d6b4-0477-45de-ad97-b824b737d6f3","resource_group_name":"myresourcegroup"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '30',
 '_aml_system_scenario_identificati

In [19]:
automl_model_name = best_run.properties['model_name']
print('Best AutoML model name: ' + automl_model_name,
      'Best AutoML model run: ' + str(best_run),
      'Best AutoML model specification: ' + str(model), sep = '\n\n')

Best AutoML model name: AutoML36449ed6930

Best AutoML model run: Run(Experiment: capstone-project-automl-experiment,
Id: AutoML_36449ed6-91de-49f6-b34b-deb0dad4489b_30,
Type: azureml.scriptrun,
Status: Completed)

Best AutoML model specification: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, tree_method='auto'))], verbose=False))], meta_learner=LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False, fit_intercept=True, intercept_scaling=1.0, l1_ratios=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', random_state=None, refit=True, scoring=<azureml.automl.runtime.stack_ensemble_base.Scorer object at

In [20]:
model.steps

[('datatransformer',
  DataTransformer(
      task='classification',
      is_onnx_compatible=False,
      enable_feature_sweeping=True,
      enable_dnn=False,
      force_text_dnn=False,
      feature_sweeping_timeout=86400,
      featurization_config=None,
      is_cross_validation=True,
      feature_sweeping_config={}
  )),
 ('stackensembleclassifier',
  StackEnsembleClassifier(
      base_learners=[('26', Pipeline(memory=None,
               steps=[('standardscalerwrapper',
                       StandardScalerWrapper(
          copy=True,
          with_mean=False,
          with_std=False
      )),
                      ('xgboostclassifier',
                       XGBoostClassifier(booster='gbtree', colsample_bytree=0.9, eta=0.01, gamma=0, grow_policy='lossguide', max_bin=255, max_depth=8, max_leaves=0, n_estimators=200, n_jobs=-1, objective='reg:logistic', problem_info=ProblemInfo(...
          iteration_timeout_param=None,
          feature_column_names=None,
          label_column_name=None,
          weight_column_name=None,
          cv_split_column_names=None,
          enable_streaming=None,
          timeseries_param_dict=None,
          gpu_training_param_dict={'processing_unit_type': 'cpu'}
      ), random_state=0, reg_alpha=2.0833333333333335, reg_lambda=2.0833333333333335, subsample=0.6, tree_method='hist'))],
               verbose=False)), ('0', Pipeline(memory=None,
               steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                      ('lightgbmclassifier',
                       LightGBMClassifier(min_data_in_leaf=20, n_jobs=-1, problem_info=ProblemInfo(
          dataset_samples=354663,
          dataset_features=81,
          dataset_classes=2,
          dataset_num_categorical=0,
          dataset_categoricals=None,
          pipeline_categoricals=None,
          dataset_y_std=None,
          dataset_uid=None,
          subsampli...
          subsampling_schedule='hyperband_clip',
          cost_mode_param=None,
          iteration_timeout_mode=0,
          iteration_timeout_param=None,
          feature_column_names=None,
          label_column_name=None,
          weight_column_name=None,
          cv_split_column_names=None,
          enable_streaming=None,
          timeseries_param_dict=None,
          gpu_training_param_dict={'processing_unit_type': 'cpu'}
      ), random_state=None))],
               verbose=False)), ('20', Pipeline(memory=None,
               steps=[('maxabsscaler', MaxAbsScaler(copy=True)),
                      ('lightgbmclassifier',
                       LightGBMClassifier(boosting_type='gbdt', colsample_bytree=0.7922222222222222, learning_rate=0.1, max_bin=270, max_depth=-1, min_child_weight=7, min_data_in_leaf=0.020697586206896556, min_split_gain=0.15789473684210525, n_estimators=200, n_jobs=-1, num_leaves=239, proble...
          iteration_timeout_param=None,
          feature_column_names=None,
          label_column_name=None,
          weight_column_name=None,
          cv_split_column_names=None,
          enable_streaming=None,
          timeseries_param_dict=None,
          gpu_training_param_dict={'processing_unit_type': 'cpu'}
      ), random_state=None, reg_alpha=0.2631578947368421, reg_lambda=0.05263157894736842, subsample=0.8415789473684211))],
               verbose=False)), ('23', Pipeline(memory=None,
               steps=[('standardscalerwrapper',
                       StandardScalerWrapper(
          copy=True,
          with_mean=False,
          with_std=False
      )),
                      ('xgboostclassifier',
                       XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.001, gamma=0.1, grow_policy='lossguide', max_bin=1023, max_depth=0, max_leaves=63, n_estimators=25, n_jobs=-1, objective='reg:logistic', problem_info=ProblemInf...
          iteration_timeout_mode=0,
          iteration_timeout_param=None,
          feature_column_names=None,
          label_column_name=None,
          weight_column_name=Non

In [36]:
#TODO: Save the best model

# Register and save the best model
automl_model_registered = remote_run.register_model(model_name='default_automl_model', tags={'Training context':'Auto ML'})

automl_model_registered.download(target_dir="outputs_automl", exist_ok=True)

'outputs_automl/model.pkl'

In [37]:
from azureml.core import Model

# List registered models
for model in Model.list(ws):
    print('Name: ', model.name, 'version: ', model.version)

    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    
    print('\n')

Name:  default_automl_model version:  2
	 Training context : Auto ML


Name:  default_hyperdrive_model version:  10


Name:  default_hyperdrive_model version:  7


Name:  default_hyperdrive_model version:  6


Name:  default_hyperdrive_model version:  5


Name:  default_hyperdrive_model version:  4


Name:  default_hyperdrive_model version:  3


Name:  default_hyperdrive_model version:  2


Name:  default_hyperdrive_model version:  1


Name:  hyperdrive_best_run.pkl version:  3


Name:  hyperdrive_best_run.pkl version:  2


Name:  hyperdrive_best_run.pkl version:  1




In [38]:
!ls -l outputs_automl

total 4083
-rwxrwxrwx 1 root root 4180457 Jul 11 15:38 model.pkl


## Model Deployment

Remember you have to deploy only one of the two models you trained. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [39]:
from azureml.core.webservice import AciWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

In [40]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                                memory_gb=4, 
                                                enable_app_insights=True,
                                                description='Predict default score using AutoML')

model = Model(ws, 'default_automl_model')

#env_deploy = Environment.get(workspace=ws, name='capstoneenv')
env_deploy = Environment.get(workspace=ws, name='capstone_project_environment')

inference_config = InferenceConfig(entry_script="automl_score.py", environment=env_deploy)

service = Model.deploy(workspace=ws, 
                       name='default-automl-v1', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

In [41]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-07-11 15:38:16+00:00 Creating Container Registry if not exists.
2021-07-11 15:38:16+00:00 Registering the environment.
2021-07-11 15:38:17+00:00 Use the existing image.
2021-07-11 15:38:17+00:00 Generating deployment configuration.
2021-07-11 15:38:18+00:00 Submitting deployment to compute.
2021-07-11 15:38:21+00:00 Checking the status of deployment default-automl-v1.Current provisioning state of AmlCompute is "Deleting"

.
2021-07-11 15:40:27+00:00 Checking the status of inference endpoint default-automl-v1.
Succeeded
ACI service creation operation finished, operation "Succeeded"


INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [42]:
print(service.get_logs())

2021-07-11T15:40:15,726308000+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-07-11T15:40:15,743228800+00:00 - nginx/run 
2021-07-11T15:40:15,740832000+00:00 - iot-server/run 
2021-07-11T15:40:15,740428900+00:00 - rsyslog/run 
rsyslogd: /azureml-envs/azureml_1b0bdc80e61f4d59929283043c512952/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-07-11T15:40:16,232483400+00:00 - iot-server/finish 1 0
2021-07-11T15:40:16,241456200+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (65)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 92
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from

In [43]:
scoring_uri = service.scoring_uri
swagger_uri = service.swagger_uri

print('Scoring URI: ', scoring_uri)
print('Swagger URI: ', swagger_uri)

Scoring URI:  http://c4fb8a4b-2a69-4ca3-bdb5-49570aba1c1e.eastus2.azurecontainer.io/score
Swagger URI:  http://c4fb8a4b-2a69-4ca3-bdb5-49570aba1c1e.eastus2.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [44]:
import requests
import json

data={"data":
  [{
    'key_value': 21,
    'edad': -1.2866063,
    'sexo': 0.0,
    'est_cvl': 4.0,
    'sit_lab': 1.0,
    'ctd_hijos': 0.0,
    'flg_sin_email': 1.0,
    'ctd_veh': 0.0,
    'tip_lvledu': 7.0,
    'total_mean_of_saldo_count_per_month': 5.666666666666667,
    'total_sum_of_saldo_count_per_month': 68,
    'total_min_of_saldo_count_per_month': 3,
    'total_max_of_saldo_count_per_month': 8,
    'total_mean_of_saldo_sum_per_month': 5.2890286,
    'total_sum_of_saldo_sum_per_month': 63.46834,
    'total_min_of_saldo_sum_per_month': 2.8037066,
    'total_max_of_saldo_sum_per_month': 7.515902,
    'total_min_of_saldo_min_per_month': 0.92711896,
    'total_mean_of_saldo_min_per_month': 0.9275547,
    'total_max_of_saldo_max_per_month': 0.97077715,
    'total_mean_of_saldo_max_per_month': 0.94196373,
    'total_min_of_condicion_min_per_month': 0,
    'total_mean_of_condicion_min_per_month': 0.0,
    'total_max_of_condicion_max_per_month': 0,
    'total_mean_of_condicion_max_per_month': 0.0,
    'total_mean_of_condicion_mean_per_month': 0.0}
    ]
  }

input_data=json.dumps(data)

with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("The output is: ",result.json())

The output is:  [1]


TODO: In the cell below, print the logs of the web service and delete the service

In [45]:
print(service.get_logs())

2021-07-11T15:40:15,726308000+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-07-11T15:40:15,743228800+00:00 - nginx/run 
2021-07-11T15:40:15,740832000+00:00 - iot-server/run 
2021-07-11T15:40:15,740428900+00:00 - rsyslog/run 
rsyslogd: /azureml-envs/azureml_1b0bdc80e61f4d59929283043c512952/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-07-11T15:40:16,232483400+00:00 - iot-server/finish 1 0
2021-07-11T15:40:16,241456200+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (65)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 92
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [34]:
service.delete()
compute_target.delete()

In [35]:
model.delete()

Current provisioning state of AmlCompute is "Deleting"

